In [ ]:
import json
import random

class Base:
    def __init__(self, name):
        self.name = name
        self.damage = 100
        self.hp = 100
        self.inventory = []
    def apply_item(self, item):
        if item["HP"] != 0:
            self.hp += item["HP"]
            if item["HP"] > 0:
                print(f"{self.name}'s HP Boost by {item['HP']}.")
            else:
                print(f"{self.name}'s HP decreased by {-item['HP']}.")        
        if item["damage"] != 0:
            self.damage += item["damage"]
            if item["damage"] > 0:
                print(f"{self.name}'s damage Boost by {item['damage']}.")
            else:
                print(f"{self.name}'s damage decreased by {-item['damage']}.")
    def use_consumable(self, consumable_name):
        for item in self.inventory:
            if item["name"].lower()==consumable_name.lower() and item["Type"].lower()=="consumable":
                self.apply_item(item)
                self.inventory.remove(item)
                print(f"{self.name} used {item['name']}.")
                break
        else:
            print(f"{consumable_name}' Not found in inventory.")
    def equip_item(self, item_type, item_name):
        for item in self.inventory:
            if item["name"].lower()==item_name.lower() and item["Type"].lower()==item_type.lower():
                self.apply_item(item)
                self.inventory.remove(item)
                print(f"{self.name} equipped {item['name']}.")
                break
        else:
            print(f"{item_type} with name '{item_name}' not found in inventory.")
    def access_inventory(self, for_healing=False):
        print("Inventory:")
        if for_healing:
            consumable=[item for item in self.inventory if item["Type"].lower()=="consumable"]
            for item in consumable:
                print(f"Name: {item['name']}, Type: {item['Type']}")
        else:
            for item in self.inventory:
                print(f"Name: {item['name']}, Type: {item['Type']}")
    def attributes(self):
        print(f"Welcome {self.name}! Choose your attributes:")


class Character(Base):
    def __init__(self, name):
        super().__init__(name)
        self.equipped_categories = {"footwear": False, "leg armor": False, "chest plate": False, "headgear": False, "weapon": False}

    def choose_attributes(self):
        super().attributes()
        self.hp += 10

    def choose_weapon(self, weapon_name):
        for item in self.inventory:
            if item["name"].lower()==weapon_name.lower() and item["Type"].lower()=="weapon":
                self.apply_item(item)
                self.inventory.remove(item)
                self.equipped_categories["weapon"]=True
                print(f"{self.name} equipped {item['name']}.")
                break
        else:
            print(f"No weapon with name '{weapon_name}' found in inventory.")

    def equip_item(self, item_type, item_name):
        for item in self.inventory:
            if item["name"].lower()==item_name.lower() and item["Type"].lower()==item_type.lower():
                self.apply_item(item)
                self.inventory.remove(item)
                self.equipped_categories[item_type.lower()]=True
                print(f"{self.name} equipped {item['name']}.")
                break
        else:
            print(f"{item_type} with name '{item_name}' not found in inventory.")
    
    def all_categories_equipped(self):
        return all(self.equipped_categories.values())


def choose_character():
    print("Choose a character:")
    print("1. Jefe")
    print("2. Mafins")
    print("3. Manchis")
    print("4. Tipiti")
    print("5. Panzon")
    print("6. Lagartijo")
    choice = input("Enter the number corresponding to your choice: ")
    characters = {
        "1": "Jefe",
        "2": "Mafins",
        "3": "Manchis",
        "4": "Tipiti",
        "5": "Panzon",
        "6": "Lagartijo"
    }
    if choice in characters:
        return Character(characters[choice])
    else:
        print("Invalid choice. Please enter a valid number.")
        return choose_character()


def load_inventory():
    with open("ItemValues.json", "r") as f:
        return json.load(f)["items"]


def display_items(items):
    for item in items:
        print(f"Name: {item['name']}")
        print(f"Description: {item['description']}")
        print(f"Damage Boost: {item['damage']}")
        print(f"HP Boost: {item['HP']}")
        print()


def battle(player, opponent):
    print(f"{player.name} is battling {opponent.name}!")
    while player.hp > 0 and opponent.hp > 0:  # Fight until one of the characters' HP reaches zero
        print("\nNext Round:")
        print(f"Player HP: {player.hp}, Opponent HP: {opponent.hp}")
        player_attack = random.randint(1, player.damage)
        opponent_attack = random.randint(1, opponent.damage)
        print(f"{player.name} attacks {opponent.name} for {player_attack} damage!")
        opponent.hp -= player_attack
        if opponent.hp <= 0:
            print(f"{opponent.name} has been defeated!")
            break
        print(f"{opponent.name} attacks {player.name} for {opponent_attack} damage!")
        player.hp -= opponent_attack
        if player.hp <= 0:
            print(f"{player.name} has been defeated!")
            break
        choice = input("\nDo you want to (i)inventory, (f)ight, or (c)consume using a consumable? (i/f/c): ")
        if choice.lower() == "i":
            player.access_inventory(for_healing=True)
            category = input("Enter the category to change the item (footwear, leg armor, chest plate, headgear, weapon): ")
            item_name = input(f"Enter the name of the {category} to equip: ")
            player.equip_item(category, item_name)
        elif choice.lower() == "h":
            player.access_inventory(for_healing=True)
            consumable_name = input("Enter the name of the consumable to use: ")
            player.use_consumable(consumable_name)
            print("\nUpdated character stats:")
            print(f"Player HP: {player.hp}, Enemy HP: {opponent.hp}")
        elif choice.lower() == "f":
            continue
        else:
            print("Invalid choice. Continuing to fight.")

    else:
        print("The battle is over!")


def main():
    selected_character = choose_character()
    selected_character.choose_attributes()

    inventory = load_inventory()
    selected_character.inventory = inventory  # Assign the entire inventory to the character's inventory

    for category in selected_character.equipped_categories.keys():
        print(f"\nChoose {category}:")
        category_items = [item for item in inventory if item["Type"].lower() == category]
        display_items(category_items)
        while not selected_character.equipped_categories[category]:
            item_name = input(f"Enter the name of the {category} to equip: ")
            selected_character.equip_item(category, item_name)

    print("\nItems equipped. Starting the fight...\n")

    opponent = Character("Lolo the Man-Eating Bunny")
    opponent.hp = 500
    opponent.damage = 100

    battle(selected_character, opponent)


if __name__ == "__main__":
    main()


Choose a character:
1. Jefe
2. Mafins
3. Manchis
4. Tipiti
5. Panzon
6. Lagartijo
Enter the number corresponding to your choice: 1
Welcome Jefe! Choose your attributes:

Choose footwear:
Name: boots
Description: Normal army boots, all purpose
Damage Boost: 0
HP Boost: 60

Name: sneakers
Description: Can be used for any occasion, just need to own it
Damage Boost: 0
HP Boost: 50

Name: cowboy boots
Description: Just as versetile as sneakers, but for the rural arena
Damage Boost: 0
HP Boost: 60

Name: huaraches
Description: can be used for anything, including walking on the moon
Damage Boost: 0
HP Boost: 80

Name: hotdog buns
Description: People give weird stares, but they just hating on your swag
Damage Boost: 0
HP Boost: 250

Name: Boots with the fur
Description: Now this is another classic, but can be taken to any club and a lucky ending occurs
Damage Boost: 0
HP Boost: 100

Enter the name of the footwear to equip: hotdog buns
Jefe's HP Boost by 250.
Jefe equipped hotdog buns.

Choose 